# Example: Image feature generation
---

This notebook is based on the code from "https://github.com/GVS-Lab/germinal_center/" by Daniel Paysan and Saradha Venkatachalapathy (2023) and serves as an illustrating example of how to automatically segment biopsy images and extract a number of hand-crafted features curated in the github repository "https://github.com/GVS-Lab/chrometrics" from those images.

Briefly, this notebook provides functionality to
1. Split multi-channel into single-channel images
2. Perform image-level intensity normalization
3. Segment nuclei using a pretrained StarDist2D model
4. Identify approximate cellular masks via nuclear boundary expansion
5. Measure cellular protein intensities
6. Measure the spatial location of each nucleus in the image



### Setting up the environment 

The first step is always to read in all required external libraries. If any of the external libraries is not yet installed in the environment used to execute the code, it needs to be installed using either conda or pip. Please refer to the respective documentations of Anaconda or pip for more information.

In [1]:
# import libraries
import sys
from pathlib import Path
from glob import glob
import pandas as pd
import os
from tqdm.notebook import tqdm

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
sys.path.append("../..")

from src.batch.nuclear_segmentation import segment_objects_stardist2d
from src.batch.extract_features import *
from src.batch.cell_segmentation import *
from src.utils.cell_type_detection import assign_cell_status
from src.utils.data_viz import visualize_segmentation_results

In addition to the external libraries, we read in a number of functions that we have defined in the ``src`` directory of this code repository. Please look at the code of these functions for a better explanation.

----

## Obtain single-channel from multi-channel images

All functions are built to work on single-channel images. Thus, we first obtain one single-channel image for each channel in the raw multi-channel inputs. This can be done via the function below. Note that you need to call that function for each channel of your multi-channel images. Note that python works on a zero-index level. So ``channel=0`` will focus on the first channel in your multi-channel image.

In [ ]:

extract_channel_save_image("path/to/multichannel/images", "path/to/images/of/first/channel", channel=0)
    
extract_channel_save_image("path/to/multichannel/images", "path/to/images/of/second/channel", channel=1)

extract_channel_save_image("path/to/multichannel/images", "path/to/images/of/third/channel", channel=2)
...


In the remainder of the notebook, we will assume that the first channel contained the DNA images.

---

## Normalize DNA images

Depending on the used setup to acquire the images, one might want to normalize the DNA intensities such that explored intensity ranges and the overall intensity distribution of each image looks similar to those of others. To this end, we recommend quantile-based normalization as it is less prone to outliers.

The normalization will simply compute the defined quantiles, e.g. the 1-percentile and 99.8-percentile, of the intensities observed in an image and subtract all intensity by the lower one, while scaling it by the difference of the two quantiles. It then rescales the intensities to a scale of 0-255 (8bit resolution) by clipping all values outside that range.

The scaled DNA images will be saved in a new directory.

In [ ]:
dna_image_dir = "/path/to/images/of/first/channel
scaled_dna_image_dir = "/path/to/normalized/images/of/first/channel"

quantile_normalize_and_save_images(dna_image_dir, scaled_dna_image_dir, mask_dir=None,
                                   quantiles=[0.01, 0.998])

---

## Segment nuclei

We next segment individual nuclei using a pretrained [StarDist](https://github.com/stardist/stardist) model. Please refer to the official StarDist documentation for more information.

In [ ]:
segment_objects_stardist2d(image_dir = scaled_dna_image_dir+"/",
                               output_dir_labels = /path/to/segmentation/masks,
                               output_dir_ijroi = "/path/to/roi/masks/for/Fiji,
                               use_pretrained = True,
                               prob_thresh=0.43)
                               normalize_quants=[0, 100])

In the above function you need to specify the location of the directory that contains the DNA images that are to be segmented as the parameter ``image_dir``. We here assume, one has previously obtained quantile-normalized images as described before. 

The parameter ``output_dir_label`` is set to the directory, where the segmentation masks i.e. images where each pixel gets a 0 if it was identified as background by the segmentation model or a number i>0, where i is the index of the nuclei.

The parameter ``output_dir_ijroi`` defines where some other form of representation of the segmented masks should be stored, which can be read in to Fiji's RoI Manager, which can be helpful for instance to easily visualize the segmentations.

The ``use_pretrained`` parameter says the function to use a pretrained model. Note that this parameter should on general not be changed.

Simarily, the ``prob_thresh`` parameter can be for most cases left as set above. However, if one is not satisfied with the segmentation quality, one could experiment with different choices (between 0 and 1) for that parameter. For an explanation of what that parameter does please refer to the official StarDist documentation linked before.

Finally, sometimes additionally enhancing the contrast of the images before segmentation can be beneficial. This could be done by setting the parameter ``norm_quants``. For values greater 0 respectively lower than 100 the contrast gets enhanced via quantile normalization, which we had described before. As a starting point, we recommend using 0 and 100 or setting the paramter to ``normalize_quants=None``.


---

## Visualization of the segmentation results

To visualize the segmentation results, one can plot the DNA image overlaid with differently colored masks for the nuclei or their corresponding outlines. This can be done as shown below.

In [ ]:
visualize_segmentation_results(
    image_dir=scaled_dna_image_dir,
    mask_dir="path/to/segmentation/masks,
    overlay_output_dir="/path/to/images/with/overlaid/mask),
    outline_output_dir="/path/to/images/with/overlaid/outline",
    alpha=0.3,
)

The parameter ``alpha`` controls the opacity of the overlaid masks but usually does not need to be adjusted.

---

## Chrometric feature extraction

We now profile the chromatin states of the identified nuclei using our developed chrometric features. This can be run in batch mode for all images as follows. The output is stored in the directory defined as the ``output_dir``.

In [ ]:
nuc_features = extract_nmco_feats_batch(
    raw_image_path=scaled_dna_image_dir + "/",
    labelled_image_path="/path/to/segmentation/masks/,
    output_dir=/path/to/chrometric/features/,
    hc_threshold=1,
    glcm_lengths=[1,5,20]
)

The last two parameters might have to be adjusted for your specific use case. The ``hc_threshold`` parameter determines the separation of each nuclei into hetero- and euchromatin based on the observed DNA intensity. Namely, heterochromatin is identified as those regions, where the observed intensity is greater than the average DNA intensity in the nucleus + ``hc_threshold`` times the standard deviation of the nuclear DNA intensities.

The choice of the hyperparameter is optimally based on respective stained images, where one can for a number of individual nuclei images check which choice of the parameter based highlights e.g. heterochromatin as marked by e.g. HP1a.

The ``glcm_lengths`` parameter determines the length scales for the GLCM texture features. Note that the units are given in pixels and the choice should be based on the sizes of nuclei that you are working with. The default values of the two parameters is based on a breast cancer study and worked well for the cell types encountered in respective tissues.

---

## Cell segmentation

To obtain cellular masks from the nuclear masks, the function below can be run. The function expands the nuclear boundaries up until a certain threshold is reached or it collides with the (expanded) boundaries of other nuclei. This can be seen as an shape-informed variant of Voronoi clustering. The second parameter is the output directory and the first one the one where the nuclear masks are stored in.

In [ ]:
cell_seg_dilation_batch(/path/to/segmentation/masks, /path/to/cell/masks/)

---

## Measuring cellular protein expression

We next quantify the expression of the cellular proteins labeled in the other channels of the multi-channel image.

In [ ]:
cd3_levels = measure_intensity_batch(
    labelled_image_path="/path/to/cell/masks/",
    protein_image_path="/path/to/cd3/channel/images",
    output_dir="/path/to/cellular/cd3_levels/dir/",
)

lamin_levels = measure_intensity_batch(
    labelled_image_path=/path/to/cell/masks/,
    protein_image_path="/path/to/lamin/channel/images",
    output_dir="/path/to/cellular/lamin_levels/dir/",
)

Here, we provide an example on how that looks assuming the one had for instance measured the expression of CD3 and Lamin

----

## Compute spatial coordinates of the cells

To enable spatial analyses, we also compute the spatial coordinates of the centroids of all segmented cells.

In [ ]:
spatial_coordiates = extract_spatial_coordinates_batch(labelled_image_path = /path/to/segmentation/masks, 
                        output_dir = /path/to/spatial_coordinates/dir/)

## Data consolidation

As a final step, we consolidate all information for all images and save it as a joint file. Note that this is only required if one wants to analyze the data from all images jointly.

In [ ]:
# Create unique nuclei IDs
nuc_features['nuc_id'] = nuc_features['image'].astype(str) + '_'+ nuc_features['label'].astype(str)
lamin_levels['nuc_id'] = lamin_levels['image'].astype(str) + '_'+ lamin_levels['label'].astype(str)
cd3_levels['nuc_id'] = cd3_levels['image'].astype(str) + '_'+ cd3_levels['label'].astype(str)

# Save the data aquired
consolidated_features_dir = "/path/to/consolidated/features/dir"
Path(consolidated_features_dir).mkdir(parents=True, exist_ok=True)

nuc_features.to_csv(consolidated_features_dir+"/" +"nuc_features.csv")
lamin_levels.to_csv(consolidated_features_dir+"/" +"lamin_levels.csv")
cd3_levels.to_csv(consolidated_features_dir+"/" +"cd3_levels.csv")
spatial_coordiates.to_csv(consolidated_features_dir+"/" +"spatial_coordiates.csv")